# 数据处理（加前缀）

In [3]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [4]:
import jsonlines
import json

In [5]:
# zh2en={'B-场景-位置':'usage location', 'B-场景-用途':'application', 'B-场景-设备':'device', 'B-场景-人':'identity'}
zh2en={'B-场景-Where':'Where', 'B-场景-Purpose':'Purpose', 'B-场景-Moment':'Moment', 'B-场景-Who':'Who'}
en2sym={'Where':'LOU', 'Purpose':'PI', 'Moment':'TOU', 'Who':'TC'}
hyp=[]
ref=[]
# en2sym={'usage location':'LOC', 'application':'USA', 'device':'DEV', 'identity':'IDE'}
RES=[]
num_dict={'B-场景-Where':0, 'B-场景-Purpose':0, 'B-场景-Moment':0, 'B-场景-Who':0}
ii=0
inp='dev'
with jsonlines.open('../shulexv1/{}.jsonl'.format(inp)) as reader:
    with open('./{}.txt'.format(inp),'w') as f:
        for obj in reader:
            res={' '.join([' ' if i =='\n' else i for i in obj['tokens']]):[]}
            for i in range(len(obj['tags'])):
                if obj['tags'][i] in ['B-场景-Where', 'B-场景-Purpose', 'B-场景-Moment', 'B-场景-Who']:
                    num_dict[obj['tags'][i]]+=1
                    r={"text": obj['tokens'][i], "start": i, "end": None, 'prompt':zh2en[obj['tags'][i]]}
                    i+=1
                    while i<len(obj['tokens']) and obj['tags'][i] in ['I-场景-Where', 'I-场景-Purpose', 'I-场景-Moment', 'I-场景-Who']:
                        r["text"]=r["text"]+' '+obj['tokens'][i]
                        i+=1
                    r['end']=i-1
#                     ref.append(zh2en[obj['tags'][i]]+' : '+' '.join([' ' if i =='\n' else i for i in obj['tokens']]))
#                     hyp.append()
                    res[' '.join([' ' if i =='\n' else i for i in obj['tokens']])].append(r)
#                     f.write(json.dumps({"content": ' '.join([' ' if i =='\n' else i for i in obj['tokens']]), "result_list": [{"text": r['text'], "start": r['start'], "end": r['end']}], "prompt": r['prompt']}))
# #                     f.write('{"content": {}, "result_list": [{"text": {}, "start": {}, "end": {}}], "prompt": {}}'.format(' '.join([' ' if i =='\n' else i for i in obj['tokens']]),r['text'],r['start'],r['end'],r['prompt']))
#                     f.write('\n')
            RES.append(res)
#             ii+=1
#             if ii==2:
#                 break

                
RES[0]

{'i bought this specifically for a 5 day vacation to colorado .': [{'text': '5 day vacation to colorado',
   'start': 6,
   'end': 10,
   'prompt': 'Purpose'}]}

In [6]:
for e in RES:
    s = list(e.keys())[0]
    e['sentence']=s
    for en in en2sym.keys():
        e[en]=[]
    for j in e[s]:
        e[j['prompt']].append(j['text'])

In [7]:

RES[1]

{'i bought this for home initially , and it worked so well that i bought it for work too .': [{'text': 'home',
   'start': 4,
   'end': 4,
   'prompt': 'Purpose'},
  {'text': 'work', 'start': 17, 'end': 17, 'prompt': 'Purpose'}],
 'sentence': 'i bought this for home initially , and it worked so well that i bought it for work too .',
 'Where': [],
 'Purpose': ['home', 'work'],
 'Moment': [],
 'Who': []}

In [122]:
hyp=[]
ref=[]
for e in RES:
    for en in en2sym.keys():
        ref.append(en+' : '+e['sentence'])
        if len(e[en])>1:
            hyp.append(en+" are "+' and '.join(e[en]))
        else:
            hyp.append(en+" is "+' '.join(e[en]) if len(e[en])>0 else en+" is "+'None')
    

In [123]:
data={'ref':ref, 'hyp':hyp}
import pandas as pd
df=pd.DataFrame(data)
df=df.applymap(str)
df.to_csv('data/shulexv2_{}.csv'.format(inp),index=False)

In [ ]:
data['hyp']

In [95]:
import pandas as pd
data = pd.read_csv('./data/shulex_dev.csv')
data.loc[122,'ref']

"device : it is in my shower and together with the teak stool , makes it feel like i 'm at a spa whenever i shower ."